In [ ]:
# Not working
# import os
# os.environ['HF_HOME'] = 'D:\Machine_Learning\sandbox\.cache\huggingface'

# %env HF_HOME='D:\Machine_Learning\sandbox\.cache'

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer, setup_chat_format
import torch

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)

# Load the model and tokenizer
model_name = "HuggingFaceTB/SmolLM2-135M"
model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_name, cache_dir="D:\Machine_Learning\sandbox\.cache"
).to(device)
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name, cache_dir="D:\Machine_Learning\sandbox\.cache")

# Set up the chat format
model, tokenizer = setup_chat_format(model=model, tokenizer=tokenizer)

# Set our name for the finetune to be saved &/ uploaded to
finetune_name = "SmolLM2-FT-MyDataset"
finetune_tags = ["smol-course", "module_1"]

d:\Machine_Learning\sandbox\pyenv3.10\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Let's test the base model before training
prompt = "Write a haiku about programming"

# Format with template
messages = [{"role": "user", "content": prompt}]
formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)

# Generate response
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=100)
print("Before training:")
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Before training:
user
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a


In [3]:
from datasets import load_dataset

# TODO: define your dataset and config using the path and name parameters
ds = load_dataset(path="HuggingFaceTB/smoltalk", name="everyday-conversations", cache_dir="D:\Machine_Learning\sandbox\.cache")

Generating test split: 100%|██████████| 119/119 [00:00<00:00, 51824.54 examples/s]


In [10]:
ds["test"]["full_topic"]

['Travel/Tourist attractions/Local markets',
 'Technology/Cameras/Camera brands',
 'shapes/symmetry',
 'Travel/Travel experiences/Luxury travel experiences',
 'Technology/Internet/Internet history',
 'basic math concepts/fractions and percentages',
 'Fitness/Team sports/Sports equipment',
 'Fashion/Shoes/High heels',
 'Health/Mental health/Work-life balance',
 'history/history and geography connections',
 'Hobbies/Painting/Art preservation',
 'Sleep/Bedtime routines/Optimal bedtime',
 'Food/Beverage varieties/Juices',
 'Entertainment/Music/Music production',
 'Education/Educational resources/Online databases',
 'colors/mixing colors',
 'Family/Siblings/Support systems',
 'nutrition/balanced diet',
 'Home/Home improvements/Window replacements',
 'Health/Mental health/Therapy options',
 'Cooking/Kitchen tools/Cooking utensils',
 'animals/endangered species',
 'light/reflection',
 'Food/Breakfast foods/Toast and spreads',
 'Cooking/Healthy cooking/Low-fat cooking methods',
 'Shopping/Budg

In [11]:
ds["test"]["messages"]

[[{'content': 'Hey!', 'role': 'user'},
  {'content': 'Hello! How can I help you today?', 'role': 'assistant'},
  {'content': "I'm planning a trip to Paris. What are some popular tourist attractions?",
   'role': 'user'},
  {'content': 'The Eiffel Tower, the Louvre Museum, and Notre Dame Cathedral are must-visit places in Paris.',
   'role': 'assistant'},
  {'content': 'That sounds great. Are there any local markets I should check out?',
   'role': 'user'},
  {'content': 'Yes, the Champs-Élysées Christmas Market and the Marché aux Puces de Saint-Ouen (flea market) are very popular among tourists and locals alike.',
   'role': 'assistant'},
  {'content': 'Awesome, thank you for the recommendations!', 'role': 'user'},
  {'content': "You're welcome! Have a great time in Paris!",
   'role': 'assistant'}],
 [{'content': 'Hi there', 'role': 'user'},
  {'content': 'Hello! How can I help you today?', 'role': 'assistant'},
  {'content': "I'm looking for a new camera. What are some popular camera

In [13]:
# Configure the SFTTrainer
sft_config = SFTConfig(
    output_dir="./sft_output",
    max_steps=1000,  # Adjust based on dataset size and desired training duration
    per_device_train_batch_size=4,  # Set according to your GPU memory capacity
    learning_rate=5e-5,  # Common starting point for fine-tuning
    logging_steps=20,  # Frequency of logging training metrics
    save_steps=100,  # Frequency of saving model checkpoints
    evaluation_strategy="steps",  # Evaluate the model at regular intervals
    eval_steps=50,  # Frequency of evaluation
    use_mps_device=(
        True if device == "mps" else False
    ),  # Use MPS for mixed precision training
    hub_model_id=finetune_name,  # Set a unique name for your model
)

d:\Machine_Learning\sandbox\pyenv3.10\lib\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [14]:
# Initialize the SFTTrainer
trainer = SFTTrainer(
    model=model,
    args=sft_config,
    train_dataset=ds["train"],
    tokenizer=tokenizer,
    eval_dataset=ds["test"],
)

C:\Users\91702\AppData\Local\Temp\ipykernel_11132\1964141755.py:2: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(
Map: 100%|██████████| 119/119 [00:00<00:00, 1534.60 examples/s]


In [15]:
# Train the model
trainer.train()

# Save the model
trainer.save_model(f"./{finetune_name}")

  0%|          | 4/1000 [00:34<2:16:16,  8.21s/it]

KeyboardInterrupt: 

In [12]:
ds_stack_smol = load_dataset(path="bigcode/the-stack-smol", cache_dir="D:\Machine_Learning\sandbox\.cache")

d:\Machine_Learning\sandbox\pyenv3.10\lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\91702\.cache\huggingface\hub\datasets--bigcode--the-stack-smol. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


DatasetNotFoundError: Dataset 'bigcode/the-stack-smol' is a gated dataset on the Hub. You must be authenticated to access it.